In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
import requests
from requests_oauthlib import OAuth1

In [5]:
import cryptography
from cryptography.fernet import Fernet

In [6]:
import datetime as dt

In [ ]:
import random

In [7]:
from Candidates import candidates_list

In [8]:
model = load_model("candidate_classifier.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
with open ('../config/config_key.key', 'rb') as ck:
    fernet_key = ck.read()

with open('../config/config_encrypt_1.key', 'rb') as c1:
    cke_e = c1.read()
with open('../config/config_encrypt_2.key', 'rb') as c2:
    cse_e = c2.read()
with open('../config/config_encrypt_3.key', 'rb') as c3:
    ate_e = c3.read()
with open('../config/config_encrypt_4.key', 'rb') as c4:
    atse_e = c4.read()

In [10]:
fernet = Fernet(fernet_key)

In [11]:
cke_d = fernet.decrypt(cke_e)
cse_d = fernet.decrypt(cse_e)
ate_d = fernet.decrypt(ate_e)
atse_d = fernet.decrypt(atse_e)

In [12]:
ck = cke_d.decode()
cs = cse_d.decode()
at = ate_d.decode()
ats = atse_d.decode()

In [13]:
# Create authorization object
auth = OAuth1(ck, cs, at, ats)

In [14]:
callback_url = "https://tweetocracy.herokuapp.com/"
payload = {
    'oauth_callback':callback_url
}

In [15]:
# Execute a POST/Auth request to twittier api to intitiate access
r = requests.post('https://api.twitter.com/oauth/request_token', auth = auth, data = payload)

In [16]:
print(f'Post Request Token URL:{r.url}')
print(f'Post Request Status:{r.status_code}')
print(f'Post Request Text: {r.text}')

Post Request Token URL:https://api.twitter.com/oauth/request_token
Post Request Status:200
Post Request Text: oauth_token=f0qEmwAAAAAA_QraAAABbJtWQY4&oauth_token_secret=7JutNCokAwiBH7DWx9gQ5E2FUoxldXJA&oauth_callback_confirmed=true


In [17]:
# Collect response information
response_output = r.text
# Relevant paramters are received as a string, separated by an '&' character
response_parameters = response_output.split("&")

In [18]:
# Store relevant response paramters in variables
oauth_token = response_parameters[0][12:]
print(f'OAuth_token:{oauth_token}')
oauth_token_secret=response_parameters[1][19:]
print(f'Oauth Token Secret:{oauth_token_secret}')
oauth_callback_confirmed = bool(response_parameters[2][25:])
print(f'Callback Confirmed:{oauth_callback_confirmed}')

OAuth_token:f0qEmwAAAAAA_QraAAABbJtWQY4
Oauth Token Secret:7JutNCokAwiBH7DWx9gQ5E2FUoxldXJA
Callback Confirmed:True


In [19]:
extended_payload = {
    'tweet_mode': 'extended'
}

In [30]:
random_candidate = random.choice(candidates_list)
candidate_id = random_candidate["twitter_user_id"]

In [31]:
user_get = requests.get(f'https://api.twitter.com/1.1/statuses/user_timeline.json?id={candidate_id}&count=100', params = extended_payload, auth = auth)


In [32]:
user_json = user_get.json()

In [33]:
# for tweet in user_json:
#     print(tweet)

In [34]:
# today_datetime = dt.datetime.utcnow()
# today_date = today_datetime.date()
# two_days_prior = dt.date.today() - dt.timedelta(days = 2)

In [35]:
# Functions for converting string date data received from Twitter API into datetime objects
# def convert_time(date_string):
#     datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
#     time_object = datetime_object.time()
#     print(time_object)
#     return time_object
# def convert_date(date_string):
#     datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
#     date_object = datetime_object.date()
#     print(date_object)
#     return date_object
# def convert_datetime(date_string):
#     datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
#     print(datetime_object)
    return datetime_object

In [36]:
def time_filter(list_element):
    date_string = list_element["created_at"]
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    date_object = datetime_object.date()
    
    today_datetime = dt.datetime.utcnow()
    today_date = today_datetime.date()
    two_days_prior = dt.date.today() - dt.timedelta(days = 2)
    
    return date_object <= two_days_prior

In [37]:
user_filtered = list(filter(lambda x: time_filter(x), user_json))

In [38]:
len(user_filtered)

89

In [39]:
tweet_selection = random.choice(user_filtered)

In [40]:
tweet_selection

{'created_at': 'Mon Aug 05 23:38:16 +0000 2019',
 'id': 1158522641474576385,
 'id_str': '1158522641474576385',
 'full_text': "[TW: Rape.]\n\nHow often do we need to read stories like this before we connect these dots?\n\nWe have to talk about the full breadth of the national crisis we're facing—gun violence, white nationalist terrorism, and violent misogyny. https://t.co/dw8AatET3J",
 'truncated': False,
 'display_text_range': [0, 254],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/dw8AatET3J',
    'expanded_url': 'https://www.cbsnews.com/news/dayton-ohio-shooter-connor-betts-kept-hit-list-and-rape-list-bellbrook-high-school-classmates-say/',
    'display_url': 'cbsnews.com/news/dayton-oh…',
    'indices': [231, 254]}]},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_use

In [41]:
tweet_dict = {
    'full_text': tweet_selection["full_text"],
    'retweet_count': tweet_selection["retweet_count"],
    'favorite_count': tweet_selection['favorite_count'],
    'created_at': tweet_selection['created_at']
}

In [42]:
def convert_day(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    day = dt.datetime.strftime(datetime_object, "%A")
    return day
def convert_hour(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    hour = dt.datetime.strftime(datetime_object, "%H")
    return hour
def convert_month(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    month = dt.datetime.strftime(datetime_object, "%B")
    return month

In [43]:
tweet_dict['day'] = convert_day(tweet_dict['created_at'])
tweet_dict['hour'] = convert_hour(tweet_dict['created_at'])
tweet_dict['month']  = convert_month(tweet_dict['created_at'])

In [44]:
tweet_dict

{'full_text': "[TW: Rape.]\n\nHow often do we need to read stories like this before we connect these dots?\n\nWe have to talk about the full breadth of the national crisis we're facing—gun violence, white nationalist terrorism, and violent misogyny. https://t.co/dw8AatET3J",
 'retweet_count': 378,
 'favorite_count': 1040,
 'created_at': 'Mon Aug 05 23:38:16 +0000 2019',
 'day': 'Monday',
 'hour': '23',
 'month': 'August'}

In [45]:
tweet_list = [tweet_dict]

In [46]:
tweet_df = pd.DataFrame(tweet_list)
# tweet_df = pd.get_dummies(tweet_df, columns = ['month', 'day', 'hour'])
tweet_df.head()

,created_at,day,favorite_count,full_text,hour,month,retweet_count
0,Mon Aug 05 23:38:16 +0000 2019,Monday,1040,[TW: Rape.]\n\nHow often do we need to read st...,23,August,378


In [47]:
import nltk
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [48]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

# NGramVectorizer
ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_text)
ngram_vect.fit_transform(tweet_df['full_text'])

<1x23 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [286]:
ngram_vect.get_feature_names()

['able',
 'american',
 'bargain',
 'basic',
 'broken',
 'country',
 'create',
 'hard',
 'helped',
 'httpstco1x8otxq3lb',
 'prosperity',
 'restore',
 'share',
 'time',
 'used',
 'worked',
 'worker']

In [211]:
X_count.toarray()

array([[1, 1, 1, 1, 1, 1]], dtype=int64)

In [92]:
# tweet_df.drop(columns = ["full_text", "created_at"], inplace = True)

In [50]:
import pickle
with open('columns.pkl', 'rb') as f:
   columns_list = pickle.load(f)

In [51]:
# columns_list

In [52]:
null_list = []
for i in range(0, len(columns_list)):
    null_list.append(0)

In [55]:
# for x in range(0, len(columns_list)):
#     print('works')

In [56]:
X_features = dict(zip(columns_list, null_list))

In [57]:
X_features['retweet_count'] = tweet_dict['retweet_count']
X_features['favorite_count'] = tweet_dict['favorite_count']

select_month = tweet_dict['month']
select_day = tweet_dict['day']
select_hour = tweet_dict['hour']
X_features[f'month_{select_month}'] = 1
X_features[f'day_{select_day}'] = 1
X_features[f'hour_{select_hour}'] = 1

In [58]:
for word in ngram_vect.get_feature_names():
    if word in X_features.keys():
        X_features[word] += 1

In [59]:
len(list(X_features.values()))

58604

In [60]:
X_features = np.array(list(X_features.values())).reshape((1, 58604))

In [61]:
X_features.shape

(1, 58604)

In [62]:
from scipy.sparse import csr_matrix

In [63]:
X_sparse = csr_matrix(X_features)

In [64]:
X_sparse

<1x58604 sparse matrix of type '<class 'numpy.int32'>'
	with 28 stored elements in Compressed Sparse Row format>

In [65]:
X_sparse.toarray()

array([[ 378, 1040,    0, ...,    0,    0,    0]], dtype=int32)

In [66]:
from sklearn.externals import joblib
scaler_filename = "mas_scaler.save"
scaler = joblib.load(scaler_filename) 

In [67]:
X_scaled = scaler.transform(X_sparse)

In [68]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.classes_ = np.load('classes.npy')

In [69]:
prediction = model.predict_classes(X_scaled)
prediction_label = encoder.inverse_transform(prediction)
prediction_label

array(['Kirsten Gillibrand'], dtype=object)

In [70]:
encoder.classes_

array(['Amy Klobuchar', 'Andrew Yang', 'Bernie Sanders', "Beto O'Rourke",
       'Bill de Blasio', 'Cory Booker', 'Donald J. Trump',
       'Elizabeth Warren', 'Gov. Bill Weld', 'Jay Inslee', 'Joe Biden',
       'Joe Sestak', 'John Delaney', 'John Hickenlooper', 'Julián Castro',
       'Kamala Harris', 'Kirsten Gillibrand', 'Marianne Williamson',
       'Michael Bennet', 'Mike Gravel', 'Pete Buttigieg',
       'Rep. Eric Swalwell', 'Seth Moulton', 'Steve Bullock', 'Tim Ryan',
       'Tom Steyer', 'Tulsi Gabbard', 'Wayne Messam'], dtype=object)

In [71]:
prediction

array([16], dtype=int64)

In [72]:
model.predict_classes(X_scaled)

array([16], dtype=int64)

In [73]:
# model.predict_proba(X_scaled)
prediction_prob = model.predict_proba(X_scaled)
prediction_prob_sort = np.sort(prediction_prob)
first_prob = prediction_prob_sort.item(-1)
first_prob
# prediction_labels = encoder.inverse_transform(prediction_prob)
# prediction_labels

1.0

In [74]:
prediction_prob

array([[5.1063542e-11, 3.3717498e-10, 2.6654059e-11, 9.7614365e-13,
        2.8052278e-11, 8.6003364e-09, 1.2584144e-19, 5.4604897e-11,
        7.6952564e-14, 1.0157901e-08, 8.6588250e-11, 2.5909721e-13,
        9.3188541e-09, 3.2402492e-10, 1.4109083e-09, 4.9031168e-10,
        1.0000000e+00, 2.0710191e-11, 1.1321262e-12, 7.9536995e-11,
        2.1025561e-09, 9.1328676e-15, 7.0547135e-10, 3.9124527e-11,
        4.4294808e-09, 9.9823461e-10, 4.4548793e-14, 5.5728015e-14]],
      dtype=float32)

In [81]:
prediction_prob = [float(i) for i in prediction_prob[0]]

In [82]:
classes_prob = list(zip(prediction_prob, encoder.classes_))
sorted_class = sorted(classes_prob, key = lambda x: x[0], reverse = True)

In [85]:
sorted_top = sorted_class[0:2]
sorted_top
type(sorted_top[0][0])

float

In [77]:
list(zip(encoder.classes_))

[('Amy Klobuchar',),
 ('Andrew Yang',),
 ('Bernie Sanders',),
 ("Beto O'Rourke",),
 ('Bill de Blasio',),
 ('Cory Booker',),
 ('Donald J. Trump',),
 ('Elizabeth Warren',),
 ('Gov. Bill Weld',),
 ('Jay Inslee',),
 ('Joe Biden',),
 ('Joe Sestak',),
 ('John Delaney',),
 ('John Hickenlooper',),
 ('Julián Castro',),
 ('Kamala Harris',),
 ('Kirsten Gillibrand',),
 ('Marianne Williamson',),
 ('Michael Bennet',),
 ('Mike Gravel',),
 ('Pete Buttigieg',),
 ('Rep. Eric Swalwell',),
 ('Seth Moulton',),
 ('Steve Bullock',),
 ('Tim Ryan',),
 ('Tom Steyer',),
 ('Tulsi Gabbard',),
 ('Wayne Messam',)]

In [224]:
help(model)

Help on Sequential in module keras.engine.sequential object:

class Sequential(keras.engine.training.Model)
 |  Sequential(layers=None, name=None)
 |  
 |  Linear stack of layers.
 |  
 |  # Arguments
 |      layers: list of layers to add to the model.
 |  
 |  # Example
 |  
 |  ```python
 |  # Optionally, the first layer can receive an `input_shape` argument:
 |  model = Sequential()
 |  model.add(Dense(32, input_shape=(500,)))
 |  
 |  # Afterwards, we do automatic shape inference:
 |  model.add(Dense(32))
 |  
 |  # This is identical to the following:
 |  model = Sequential()
 |  model.add(Dense(32, input_dim=500))
 |  
 |  # And to the following:
 |  model = Sequential()
 |  model.add(Dense(32, batch_input_shape=(None, 500)))
 |  
 |  # Note that you can also omit the `input_shape` argument:
 |  # In that case the model gets built the first time you call `fit` (or other
 |  # training and evaluation methods).
 |  model = Sequential()
 |  model.add(Dense(32))
 |  model.add(Dense(32